<a href="https://colab.research.google.com/github/JeJe620/NYU-SUMMER-2021/blob/main/2SLS_section.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Briefings**

1. Two packages:


*   https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.IV2SLS.html
*   https://bashtage.github.io/linearmodels/doc/iv/examples/advanced-examples.html

2. Four variables:

Dependent Variables: Y
Endogenous Variable: X
Control variables: W
instruments: Z

3. Two regressions


*   First: X~Z
*   Second: Y~(X+W)





background readings: https://cryptobriefing.com/earning-20-apy-with-yield-app/

# Data Preparations

### Date Sources 
https://docs.defipulse.com/overview-of-api-endpoints

https://data-beta.defipulse.com/dashboard/egs

https://docs.defipulse.com/api-docs-by-provider/defi-pulse-data/total-value-locked/total-value-locked

### Import the Historical Data of Total Value Locked

In [ ]:
import pandas as pd
import requests
import json 
from datetime import datetime
TVL= requests.get('https://data-api.defipulse.com/api/v1/defipulse/api/GetHistory?resolution=history&project=all&api-key=10095a3e336c5beb50c0e30ef3ab8ebdcde3e0c08f65f3ae462f69a6902a')
TVL= TVL.json()
TVL=pd.DataFrame(TVL)
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI
0,1624687200,48437808290,2.634695e+07,163787.898732,8.760566e+06,2.699045e+09
1,1624665600,47965626508,2.641104e+07,163732.474282,8.746788e+06,2.701426e+09
2,1624579200,50994717087,2.560618e+07,164570.777150,8.682699e+06,2.692037e+09
3,1624492800,50101916045,2.539506e+07,162681.342310,8.509900e+06,2.663994e+09
4,1624406400,48438860331,2.578373e+07,150849.363021,8.494428e+06,2.510972e+09


### Epoch & Unix Timestamp Conversion Tools: 

https://www.epochconverter.com/

In [ ]:
import datetime
TVL['timestamp']=TVL['timestamp'].astype(int)
TVL.dtypes

timestamp      int64
tvlUSD         int64
tvlETH       float64
BTC          float64
ETH          float64
DAI          float64
dtype: object

In [ ]:
TVL['time']=TVL['timestamp'].apply(lambda x: datetime.datetime.utcfromtimestamp(x).replace(tzinfo=datetime.timezone.utc))
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,time
0,1624687200,48437808290,2.634695e+07,163787.898732,8.760566e+06,2.699045e+09,2021-06-26 06:00:00+00:00
1,1624665600,47965626508,2.641104e+07,163732.474282,8.746788e+06,2.701426e+09,2021-06-26 00:00:00+00:00
2,1624579200,50994717087,2.560618e+07,164570.777150,8.682699e+06,2.692037e+09,2021-06-25 00:00:00+00:00
3,1624492800,50101916045,2.539506e+07,162681.342310,8.509900e+06,2.663994e+09,2021-06-24 00:00:00+00:00
4,1624406400,48438860331,2.578373e+07,150849.363021,8.494428e+06,2.510972e+09,2021-06-23 00:00:00+00:00


In [ ]:
TVL['date'] = pd.to_datetime(TVL['time']).dt.date
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,time,date
0,1624687200,48437808290,2.634695e+07,163787.898732,8.760566e+06,2.699045e+09,2021-06-26 06:00:00+00:00,2021-06-26
1,1624665600,47965626508,2.641104e+07,163732.474282,8.746788e+06,2.701426e+09,2021-06-26 00:00:00+00:00,2021-06-26
2,1624579200,50994717087,2.560618e+07,164570.777150,8.682699e+06,2.692037e+09,2021-06-25 00:00:00+00:00,2021-06-25
3,1624492800,50101916045,2.539506e+07,162681.342310,8.509900e+06,2.663994e+09,2021-06-24 00:00:00+00:00,2021-06-24
4,1624406400,48438860331,2.578373e+07,150849.363021,8.494428e+06,2.510972e+09,2021-06-23 00:00:00+00:00,2021-06-23


In [ ]:
TVL=TVL.drop(columns='time')
TVL['date']=pd.to_datetime(TVL['date'])
TVL.dtypes

timestamp             int64
tvlUSD                int64
tvlETH              float64
BTC                 float64
ETH                 float64
DAI                 float64
date         datetime64[ns]
dtype: object

### Import the data for Ethereum Prices

[Data Metrics](https://github.com/coinmetrics-io/data/blob/master/csv/metrics.csv)

In [ ]:
df_eth=pd.read_csv("https://raw.githubusercontent.com/coinmetrics-io/data/master/csv/eth.csv",usecols=(["time","PriceUSD","VtyDayRet30d","TxTfrValAdjUSD","AdrBalCnt","NVTAdj90","SplyCur"]))
df_eth['time']=pd.to_datetime(df_eth['time'])
df_eth.dtypes

time              datetime64[ns]
AdrBalCnt                float64
NVTAdj90                 float64
PriceUSD                 float64
SplyCur                  float64
TxTfrValAdjUSD           float64
VtyDayRet30d             float64
dtype: object

### Merge the two Data Frame

In [ ]:
df =df_eth.merge(TVL, left_on='time', right_on='date')
df.head()

,time,AdrBalCnt,NVTAdj90,PriceUSD,SplyCur,TxTfrValAdjUSD,VtyDayRet30d,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,date
0,2017-08-02,2563959.0,20.362229,217.584664,9.374206e+07,6.780871e+08,0.095886,1501632000,0,0.000000,0.0,0.0,0.0,2017-08-02
1,2017-08-03,2583277.0,20.979627,224.028898,9.376303e+07,6.062566e+08,0.096060,1501718400,4,0.018339,0.0,0.0,0.0,2017-08-03
2,2017-08-04,2605667.0,20.714810,221.471094,9.378433e+07,5.206240e+08,0.096060,1501804800,4,0.017826,0.0,0.0,0.0,2017-08-04
3,2017-08-05,2624386.0,23.205816,248.939666,9.380552e+07,6.665590e+08,0.098653,1501891200,5,0.022665,0.0,0.0,0.0,2017-08-05
4,2017-08-06,2640032.0,24.667311,266.031146,9.382679e+07,9.229966e+08,0.097469,1501977600,5,0.019756,0.0,0.0,0.0,2017-08-06


In [ ]:
df=df[["date","tvlUSD","VtyDayRet30d","SplyCur","AdrBalCnt"]]

# Test Instrument Validities

Y: "SplyCur"

X: "VtyDayRet30d"

Z: "SplyCur"

W: "AdrBalCnt"

In [ ]:
df[["VtyDayRet30d","SplyCur"]].corr()

,VtyDayRet30d,SplyCur
VtyDayRet30d,1.000000,-0.101702
SplyCur,-0.101702,1.000000


# Using the Linear Model Package

https://bashtage.github.io/linearmodels/doc/iv/index.html

In [ ]:
pip install linearmodels

     |████████████████████████████████| 1.5MB 3.1MB/s 
     |████████████████████████████████| 9.5MB 21.6MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
from linearmodels import IV2SLS

### run an OLS

In [ ]:
df['const'] = 1

In [ ]:
ivolsmod = IV2SLS(df.tvlUSD, df[["const","VtyDayRet30d","SplyCur","AdrBalCnt"]], None, None)
res_ols = ivolsmod.fit()
print(res_ols)

                            OLS Estimation Summary                            
Dep. Variable:                 tvlUSD   R-squared:                      0.7385
Estimator:                        OLS   Adj. R-squared:                 0.7380
No. Observations:                1423   F-statistic:                    1460.9
Date:                Sat, Jun 26 2021   P-value (F-stat)                0.0000
Time:                        07:25:18   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
const         7.003e+11  2.508e+10     27.919     0.0000   6.511e+11   7.495e+11
VtyDayRet30d  7.566e+10  1.317e+10     5.746

### run an IV2SLS

In [ ]:
ivmod = IV2SLS(df.tvlUSD, df[["const","AdrBalCnt"]], df["VtyDayRet30d"], df["SplyCur"])
res_2sls = ivmod.fit()
print(res_2sls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                 tvlUSD   R-squared:                     -3.3208
Estimator:                    IV-2SLS   Adj. R-squared:                -3.3269
No. Observations:                1423   F-statistic:                    238.39
Date:                Sat, Jun 26 2021   P-value (F-stat)                0.0000
Time:                        07:25:18   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
const        -1.154e+11  1.229e+10    -9.3888     0.0000  -1.395e+11  -9.132e+10
AdrBalCnt        856.60     59.331     14.43

### comparison of the results

In [ ]:
from collections import OrderedDict
from linearmodels.iv.results import compare

In [ ]:
res = OrderedDict()
res['OLS'] = res_ols
res['2SLS'] = res_2sls
print(compare(res))

                 Model Comparison                
                               OLS           2SLS
-------------------------------------------------
Dep. Variable               tvlUSD         tvlUSD
Estimator                      OLS        IV-2SLS
No. Observations              1423           1423
Cov. Est.                   robust         robust
R-squared                   0.7385        -3.3208
Adj. R-squared              0.7380        -3.3269
F-statistic                 1460.9         238.39
P-value (F-stat)            0.0000         0.0000
==================     ===========   ============
const                    7.003e+11     -1.154e+11
                          (27.919)      (-9.3888)
VtyDayRet30d             7.566e+10      1.911e+12
                          (5.7469)       (8.3447)
SplyCur                    -7595.4               
                         (-28.485)               
AdrBalCnt                   3672.8         856.60
                          (30.814)       (14.438)


### Testing endogeneity

The Durbin test is a classic of endogeneity which compares OLS estimates with 2SLS and exploits the fact that OLS estimates will be relatively efficient. Durbin’s test is not robust to heteroskedasticity.

In [ ]:
res_2sls.durbin()

Durbin test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 656.5428
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f1496c7b910

Wooldridge’s regression-based test of exogeneity is robust to heteroskedasticity since it inherits the covariance estimator from the model. Here there is little difference.

In [ ]:
res_2sls.wooldridge_regression

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 811.3742
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f1495bf84d0

### print first stage regression

In [ ]:
print(res_2sls.first_stage)

     First Stage Estimation Results    
                           VtyDayRet30d
---------------------------------------
R-squared                        0.0589
Partial R-squared                0.0552
Shea's R-squared                 0.0552
Partial F-statistic              72.210
P-value (Partial F-stat)         0.0000
Partial F-stat Distn            chi2(1)
========================== ============
const                            0.4444
                               (9.6514)
AdrBalCnt                     1.534e-09
                               (8.0798)
SplyCur                      -4.138e-09
                              (-8.4976)
---------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


# Using the Statsmodel Package

Not recommended

https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.IV2SLS.html